# Getting started with CARTOframes

## 1. Connect to a CARTO account

In [92]:
import cartoframes
from cartoframes import Credentials

creds = Credentials(username='public', key='default_public')
cc = cartoframes.CartoContext(creds=creds)

## 2. List sample datasets

In [93]:
[table.name for table in cc.tables()]

['foottraffic_nyc_24h',
 'foottraffic_atlanta_24h',
 'listings_bcn',
 'world_commerce',
 'estacion_bicicleta',
 'ne_50m_admin_0_countries',
 'manzana',
 'lineas_metro',
 'listings_madrid',
 'europe_countries',
 'quercus',
 'europe_cities',
 'barcelona_building_footprints',
 'baltimore_economic_data',
 'world_population_2015',
 'berlinschoolsspeed',
 'vegetation',
 'lakes_and_swamps',
 'mountains',
 'ocean_game_of_thrones',
 'table_5yr_county_acs',
 'ne_10m_populated_places_simple_7',
 'car_crash_data',
 'continent_points',
 'table_311_complaints_1',
 'us_states',
 'zika_cases_us_cdc',
 'stack_dummy',
 'state_points',
 'state_bounds',
 'crime_incident_reports',
 'pacific_islands_coms',
 'equation_and_180',
 'pacific_islands',
 'populated_places',
 'ne_50m_admin_1_states_1',
 'insert_polygons',
 'world_borders_hd',
 'ne_10m_populated_places_simple',
 'stormevents_locations_2014']

## 3. Read one of the sample datasets into a pandas DataFrame

In [94]:
df = cc.read('world_ports')
df.head()

,anch_depth,carbchmoor,cargmdmoor,cargo_anch,cargodepth,cargowharf,caricemoor,chan_depth,chart,comm_air,...,serv_steam,shelter,sscc_cert,the_geom,tide_range,tug_assist,tugsalvage,turn_basin,us_rep,water
cartodb_id,,,,,,,,,,,,,,,,,,,,,
31,G,None,None,Y,J,Y,None,J,97157,None,...,None,G,None,0101000020E610000077777777777F61405555555555D5...,9,Y,None,None,N,Y
183,M,None,None,Y,J,None,None,L,95103,Y,...,None,G,None,0101000020E61000002222222222A25F402222222222C2...,2,Y,None,None,None,Y
222,H,None,Y,None,None,None,None,A,57080,None,...,None,P,None,0101000020E6100000CDCCCCCCCCCCF0BFCDCCCCCCCCCC...,0,Y,None,None,None,None
410,A,None,None,None,N,None,None,N,19402,None,...,None,P,None,0101000020E61000001111111111C964C0DEDDDDDDDDDD...,1,None,None,None,None,Y
715,G,None,None,None,L,Y,None,N,54041,Y,...,None,G,None,0101000020E61000002222222222223140ABAAAAAAAA8A...,1,Y,None,None,None,Y


## 4. Create a map

In [95]:
from cartoframes.contrib import vector
vector.vmap([
  vector.Layer('world_ports', strokeWidth=0.5, strokeColor='black', size=3, color='#0ab96b')
], context=cc)